# Classify images into just two classes: "coffee" and "toast" : Transfer Learning with Fine-Tuning


`#pytorch` `#xgboost` `#embeddings` `#transfer-learning` `#resnet` `#convolutions` `#vectorization` `#resnet-18` `#resnet-50` `#fine-tuning` `full-fine-tuning`

> Objectives
>
> - Fine-tune a pretrained model to accomplish a more specific task



## Imports


In [9]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights

## Loading and transforming training data


In [10]:
# Define transformations
transform = transforms.Compose(
    [
        # Resize images to 224x224 as required by ResNet18
        transforms.Resize((224, 224)),
        # Convert the raw image data into a tensor instance
        # (a multi-dimensional matrix)
        transforms.ToTensor(),
        # Standardize images to account for outliers
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[
                0.229,
                0.224,
                0.225,
            ],  # These values are specific to the ResNet18 model
        ),
    ]
)

In [11]:
data = datasets.ImageFolder(
    "./downloads/Dataset_Example", transform=transform
)

## Train test split


In [12]:
# 70% of data is used for training
train_size = int(0.7 * len(data))
# 15% of data is used for validation
val_size = int(0.15 * len(data))
# The remaining 15% is used for testing
test_size = len(data) - train_size - val_size

train_data, val_data, test_data = random_split(
    data, [train_size, val_size, test_size]
)

## Process data in Dataloader


In [13]:
# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

## Loading and preparing the Resnet model:


In [6]:
model = resnet18(weights=ResNet18_Weights.DEFAULT, progress=False)

In [7]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # enable binary classification

## Define optimizer and loss function

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
model = model.to(device)

Using mps device


## Common fucntion to claculate accuracy


In [15]:
def calculate_accuracy(loader):
    correct = 0
    total = 0
    # Set model to evaluation mode
    model.eval()
    # Disable gradient calculation (unnecessary for inference)
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

## calculate the accuracy without fine training for validation set

In [16]:
val_accuracy = calculate_accuracy(val_loader)
print(f"Validation accuracy before fine-tuning: {val_accuracy}%")

Validation accuracy before fine-tuning: 64.20308483290488%


## Train the model with training data, a.k.a - Fine tune

In [17]:
# Number of passes through the training data
num_epochs = 2

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()

    # Iterate over data
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    val_accuracy = calculate_accuracy(val_loader)
    print(
        f"Epoch {epoch+1}/{num_epochs}, Validation accuracy: {val_accuracy}%"
    )

/Users/prasunjit/deep-learning/.venv/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch 1/2, Validation accuracy: 88.75321336760925%
Epoch 2/2, Validation accuracy: 91.58097686375321%


You should notice a huge jump in validation accuracy, now approaching or surpassing 90%. 

- [ ] Once the fine-tuning is complete, we can assess the model by calculating the accuracy for each dataset.


In [18]:
# Evaluate on all sets
train_accuracy = calculate_accuracy(train_loader)
val_accuracy = calculate_accuracy(val_loader)
test_accuracy = calculate_accuracy(test_loader)

print(f"Train accuracy: {train_accuracy}%")
print(f"Validation accuracy: {val_accuracy}%")
print(f"Test accuracy: {test_accuracy}%")

/Users/prasunjit/deep-learning/.venv/lib/python3.11/site-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train accuracy: 93.29293485745765%
Validation accuracy: 91.58097686375321%
Test accuracy: 92.48554913294798%
